# Agents Series - Episode 3
_Routing agents_

# Installs

In [0]:
%pip install langgraph langchain-openai==0.3.0

In [0]:
dbutils.library.restartPython()

# Setup

In [0]:
dbutils.widgets.text("transcript", "Hello, I would like to book a  a car, please can you help me?")

In [0]:
import os

In [0]:
os.environ["AZURE_OPENAI_ENDPOINT"] = dbutils.secrets.get(<<SCOPE>>, <<KEY>>)
os.environ["AZURE_OPENAI_API_KEY"] = dbutils.secrets.get(<<SCOPE>>, <<KEY>>)

## LLM

In [0]:
from langchain_openai import AzureChatOpenAI

In [0]:
llm = AzureChatOpenAI(
    model_name="gpt4o",
    openai_api_version="2024-08-01-preview",
    temperature= 0.9
)

# Prompt & Route

In [0]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [0]:
messages = ChatPromptTemplate.from_messages(
    [
        (
            "assistant",
            """
            You are a call centre assistant who directs customers to the best operator.
            When given a user transcript, decide whether it is best for them to talk to sales or bookings.
            Format your decision as a JSON with the following structure:
            {{
                "decision": "sales" | "bookings",
                "reason": "reason for decision"
            }}
            """
        ),
        (
            "user",
            "{transcript}"
        )
    ]
)

In [0]:
routing_chain = (
    {
        "transcript": RunnablePassthrough()
    }
    | messages
    | llm
    | JsonOutputParser()
)

In [0]:
routing = routing_chain.invoke(dbutils.widgets.get("transcript"))

In [0]:
if routing["decision"] == "sales":
    print("Redirecting to sales")
else:
    print("Redirecting to bookings")

print(f"\nThis was done because: {routing.get("reason")}")